In [2]:
import pandas as pd
import os
from PIL import Image
from keras.utils import img_to_array, load_img
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

df = pd.DataFrame(columns=['imagePath', 'label'])

directories = {
    "ayam_goreng": "./dataset/ayam_goreng/",
    "ayam_pop": "./dataset/ayam_pop/",
    "daging_rendang": "./dataset/daging_rendang/",
    "dendeng_batokok": "./dataset/dendeng_batokok/",
    "gulai_ikan": "./dataset/gulai_ikan/",
    "gulai_tambusu": "./dataset/gulai_tambusu/",
    "telur_balado": "./dataset/telur_balado/",
    "telur_dadar": "./dataset/telur_dadar/",
    "tahu": "./dataset/tahu/",
    "daun_singkong": "./dataset/daun_singkong/",
    "nangka": "./dataset/nangka/",
    "perkedel": "./dataset/perkedel/",
    "nasi" : "./dataset/nasi/"
}

for label, directory in directories.items():
    for i in os.listdir(directory):
        df = pd.concat([df, pd.DataFrame({'imagePath': [f"{directory}/{i}"], 'label': [label]})])

df = df.reset_index(drop=True)

imgs = []

for image_path in df['imagePath']:
    img = load_img(image_path, target_size=(127, 127)).convert('RGB')
    img_array = img_to_array(img) / 128
    imgs.append(img_array)

df['img'] = imgs

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.25,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
x = np.array(df['img'].tolist())
y = le.fit_transform(df['label'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
datagen.fit(x_train)

In [4]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load the VGG16 base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(127, 127, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers on top of VGG16
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(directories), activation='softmax')(x)

# Combine the base model and custom layers
model3 = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model3.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Train the model
history = model3.fit(datagen.flow(x_train, y_train, batch_size=32),
                    validation_data=(x_test, y_test),
                    epochs=50,
                    callbacks=[early_stopping, reduce_lr])

# Evaluate the model
loss, accuracy = model3.evaluate(x_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/50
67/67 [==============================] - 20s 150ms/step - loss: 2.7246 - accuracy: 0.0931 - val_loss: 2.4394 - val_accuracy: 0.2505 - lr: 0.0010
Epoch 2/50
67/67 [==============================] - 7s 102ms/step - loss: 2.5340 - accuracy: 0.1273 - val_loss: 2.3296 - val_accuracy: 0.2598 - lr: 0.0010
Epoch 3/50
67/67 [==============================] - 7s 99ms/step - loss: 2.4667 - accuracy: 0.1455 - val_loss: 2.2623 - val_accuracy: 0.2673 - lr: 0.0010
Epoch 4/50
67/67 [==============================] - 7s 105ms/step - loss: 2.3769 - accuracy: 0.1741 - val_loss: 2.0614 - val_accuracy: 0.3065 - lr: 0.0010
Epoch 5/50
67/67 [==============================] - 7s 103ms/step - loss: 2.3568 - accuracy: 0.1886 - val_loss: 2.0927 - val_accuracy: 0.2748 - lr: 0.0010
Epoch 6/50
67/67 [==============================] - 7s 103ms/step - loss: 2.3369 - accuracy: 0.1848 - val_loss: 2.0197 - val_accuracy: 0.3178 - lr: 0.0010
Epoch 7/50
67/67 [==============================] - 7s 101ms/step - lo